In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive

/content/drive


In [3]:
cd MyDrive

/content/drive/MyDrive


In [4]:
cd Data

/content/drive/MyDrive/Data


In [5]:
ls

 angle_gait.csv              eye_open/           non_PRC.csv     titanic_train.csv
'Clinical Assessment.xlsx'   final_merged.csv    par_logit.csv   weather.csv
 data.csv                    looking_back/       PRC.csv         머신러닝/
 eye_close/                  non_par_logit.csv   stand/


In [6]:
import pandas as pd
non_PRC = pd.read_csv('looking_back/look.csv')
PRC = pd.read_csv('looking_back/look_par.csv')
clinical_assessment = pd.read_excel('Clinical Assessment.xlsx')

In [7]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
import warnings
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

In [8]:
warnings.filterwarnings(action='ignore')

In [9]:
score = clinical_assessment.iloc[:,9]

In [10]:
score = score.dropna()
del score[0]
del score[1]
del score[9]
del score[18]

In [11]:
y = []
for value in score:
    if value >= 45:
        y.append(1)
    else:
        y.append(0)
print(y)

[1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0]


## Non-paretic Side

In [12]:
df_non = pd.DataFrame(non_PRC)

In [13]:
y = pd.DataFrame(y)

In [14]:
non = pd.concat([y,df_non],axis=1)

In [15]:
colname = list(non.columns)
colname[0] = "score"
non.columns = colname
non.columns

Index(['score', 'Unnamed: 0', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7',
       'PC8', 'PC9', 'PC10', 'PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16',
       'PC17', 'PC18', 'PC19', 'PC20', 'PC21', 'PC22', 'PC23', 'PC24', 'PC25',
       'PC26', 'PC27', 'PC28', 'PC29', 'PC30', 'PC31', 'PC32', 'PC33', 'PC34',
       'PC35', 'PC36', 'PC37', 'PC38', 'PC39', 'PC40', 'PC41'],
      dtype='object')

In [16]:
X = non.loc[:,['PC4','PC8','PC14','PC16','PC19','PC28']]
y = non['score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=123, stratify=y)

### logistic regression

In [17]:
lr_model = LogisticRegression()

# 선택된 특성으로 모델 학습
lr_model.fit(X_train, y_train)

y_pred_lr = lr_model.predict(X_test)

conf_matrix = confusion_matrix(y_test, y_pred_lr)
print("Confusion Matrix:\n", conf_matrix)
# 정확도 계산
accuracy_lr = accuracy_score(y_test, y_pred_lr)
print("Test Accuracy:", accuracy_lr)

# f1-score 계산
f1_lr = f1_score(y_test, y_pred_lr)
print("f1-score:", f1_lr)

# auc 계산
auc_lr = roc_auc_score(y_test,y_pred_lr)
print("AUC:",auc_lr)

# 분류 보고서 출력
classification_rep_lr = classification_report(y_test, y_pred_lr)
print("Classification Report:\n", classification_rep_lr)

Confusion Matrix:
 [[2 3]
 [3 9]]
Test Accuracy: 0.6470588235294118
f1-score: 0.75
AUC: 0.575
Classification Report:
               precision    recall  f1-score   support

           0       0.40      0.40      0.40         5
           1       0.75      0.75      0.75        12

    accuracy                           0.65        17
   macro avg       0.57      0.57      0.58        17
weighted avg       0.65      0.65      0.65        17



### Random Forest

In [18]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier


param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}


grid_search = GridSearchCV(RandomForestClassifier(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)


best_rf_model = grid_search.best_estimator_
best_params = grid_search.best_params_
print("Best Random Forest Model:", best_rf_model)
print("Best Hyperparameters:", best_params)

y_pred_best_rf = best_rf_model.predict(X_test)


conf_matrix_best = confusion_matrix(y_test, y_pred_best_rf)
print("Confusion Matrix:\n", conf_matrix_best)

accuracy_best_rf = accuracy_score(y_test, y_pred_best_rf)
print("Test Accuracy with Best Model:", accuracy_best_rf)

f1_best_rf = f1_score(y_test, y_pred_best_rf)
print("f1-score with Best Model:", f1_best_rf)

auc_best_rf = roc_auc_score(y_test, y_pred_best_rf)
print("AUC with Best Model:", auc_best_rf)

classification_rep_best_rf = classification_report(y_test, y_pred_best_rf)
print("Classification Report with Best Model:\n", classification_rep_best_rf)

Best Random Forest Model: RandomForestClassifier(min_samples_leaf=4, random_state=123)
Best Hyperparameters: {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 100}
Confusion Matrix:
 [[ 0  5]
 [ 0 12]]
Test Accuracy with Best Model: 0.7058823529411765
f1-score with Best Model: 0.8275862068965517
AUC with Best Model: 0.5
Classification Report with Best Model:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.71      1.00      0.83        12

    accuracy                           0.71        17
   macro avg       0.35      0.50      0.41        17
weighted avg       0.50      0.71      0.58        17



### SVM Linear

In [19]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear'],
    'degree':[2,3,4]
}


grid_search = GridSearchCV(SVC(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)


best_svm_model = grid_search.best_estimator_
best_params = grid_search.best_params_
print("Best SVM Model:", best_svm_model)
print("Best Hyperparameters:", best_params)


y_pred_best_svm_linear = best_svm_model.predict(X_test)


conf_matrix_best = confusion_matrix(y_test, y_pred_best_svm_linear)
print("Confusion Matrix:\n", conf_matrix_best)

accuracy_best_svm_linear = accuracy_score(y_test, y_pred_best_svm_linear)
print("Test Accuracy with Best Model:", accuracy_best_svm_linear)

f1_best_svm_linear = f1_score(y_test, y_pred_best_svm_linear)
print("f1-score with Best Model:", f1_best_svm_linear)

y_scores_best_svm = best_svm_model.decision_function(X_test)
auc_best_svm_linear = roc_auc_score(y_test, y_scores_best_svm)
print("AUC with Best Model:", auc_best_svm_linear)

classification_rep_best_svm_linear = classification_report(y_test, y_pred_best_svm_linear)
print("Classification Report with Best Model:\n", classification_rep_best_svm_linear)

Best SVM Model: SVC(C=1, degree=2, kernel='linear', random_state=123)
Best Hyperparameters: {'C': 1, 'degree': 2, 'kernel': 'linear'}
Confusion Matrix:
 [[ 1  4]
 [ 2 10]]
Test Accuracy with Best Model: 0.6470588235294118
f1-score with Best Model: 0.7692307692307692
AUC with Best Model: 0.5666666666666667
Classification Report with Best Model:
               precision    recall  f1-score   support

           0       0.33      0.20      0.25         5
           1       0.71      0.83      0.77        12

    accuracy                           0.65        17
   macro avg       0.52      0.52      0.51        17
weighted avg       0.60      0.65      0.62        17



### SVM Poly

In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['poly'],
    'degree': [2, 3, 4],
}


grid_search = GridSearchCV(SVC(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)


best_svm_model = grid_search.best_estimator_
best_params = grid_search.best_params_
print("Best SVM Model:", best_svm_model)
print("Best Hyperparameters:", best_params)


y_pred_best_svm_poly = best_svm_model.predict(X_test)


conf_matrix_best = confusion_matrix(y_test, y_pred_best_svm_poly)
print("Confusion Matrix:\n", conf_matrix_best)

accuracy_best_svm_poly = accuracy_score(y_test, y_pred_best_svm_poly)
print("Test Accuracy with Best Model:", accuracy_best_svm_poly)

f1_best_svm_poly = f1_score(y_test, y_pred_best_svm_poly)
print("f1-score with Best Model:", f1_best_svm_poly)

y_scores_best_svm_poly = best_svm_model.decision_function(X_test)
auc_best_svm_poly = roc_auc_score(y_test, y_scores_best_svm_poly)
print("AUC with Best Model:", auc_best_svm_poly)

classification_rep_best_svm_poly = classification_report(y_test, y_pred_best_svm_poly)
print("Classification Report with Best Model:\n", classification_rep_best_svm_poly)

Best SVM Model: SVC(C=0.1, degree=2, kernel='poly', random_state=123)
Best Hyperparameters: {'C': 0.1, 'degree': 2, 'kernel': 'poly'}
Confusion Matrix:
 [[ 0  5]
 [ 0 12]]
Test Accuracy with Best Model: 0.7058823529411765
f1-score with Best Model: 0.8275862068965517
AUC with Best Model: 0.6166666666666667
Classification Report with Best Model:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.71      1.00      0.83        12

    accuracy                           0.71        17
   macro avg       0.35      0.50      0.41        17
weighted avg       0.50      0.71      0.58        17



### XGBoost

In [21]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb


param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2],
}


grid_search = GridSearchCV(xgb.XGBClassifier(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)


best_xgb_model = grid_search.best_estimator_
best_params = grid_search.best_params_
print("Best XGBoost Model:", best_xgb_model)
print("Best Hyperparameters:", best_params)


y_pred_best_xgb = best_xgb_model.predict(X_test)

conf_matrix_best = confusion_matrix(y_test, y_pred_best_xgb)
print("Confusion Matrix:\n", conf_matrix_best)

accuracy_best_xgb = accuracy_score(y_test, y_pred_best_xgb)
print("Test Accuracy with Best Model:", accuracy_best_xgb)

f1_best_xgb = f1_score(y_test, y_pred_best_xgb)
print("f1-score with Best Model:", f1_best_xgb)

auc_best_xgb = roc_auc_score(y_test, y_pred_best_xgb)
print("AUC with Best Model:", auc_best_xgb)

classification_rep_best_xgb = classification_report(y_test, y_pred_best_xgb)
print("Classification Report with Best Model:\n", classification_rep_best_xgb)

Best XGBoost Model: XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=123, ...)
Best Hyperparameters: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100}
Confusion Matrix:
 [[ 0  5]
 [ 2 10]]
Test Accuracy with Best Model: 0.5882352941176471
f1-score with Best Model: 0.7407407407407408
AUC with Be

## paretic-Side

In [22]:
df_par = pd.DataFrame(PRC)
y = pd.DataFrame(y)
par = pd.concat([y,df_par],axis=1)
colname = list(par.columns)
colname[0] = "score"

In [23]:
import random
random.seed(1)
X = par.loc[:,['PC1','PC4','PC7','PC12','PC25','PC29','PC31','PC32']]
y = par['score']
X_train , X_test , y_train , y_test  = train_test_split(X,y,test_size = 0.4,random_state=123,stratify=y)

### logistic regression

In [24]:
lr_model = LogisticRegression()

lr_model.fit(X_train, y_train)

y_pred_par_lr = lr_model.predict(X_test)

conf_matrix = confusion_matrix(y_test, y_pred_par_lr)
print("Confusion Matrix:\n", conf_matrix)
# 정확도 계산
accuracy_lr_ps = accuracy_score(y_test, y_pred_par_lr)
print("Test Accuracy:", accuracy_lr_ps)

# f1-score 계산
f1_lr_ps = f1_score(y_test, y_pred_par_lr)
print("f1-score:", f1_lr_ps)

# AUC 계산
auc_lr_ps = roc_auc_score(y_test,y_pred_par_lr)
print("AUC:",auc_lr_ps)


classification_rep_lr = classification_report(y_test, y_pred_par_lr)
print("Classification Report:\n", classification_rep_lr)

Confusion Matrix:
 [[0 5]
 [4 8]]
Test Accuracy: 0.47058823529411764
f1-score: 0.64
AUC: 0.3333333333333333
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.62      0.67      0.64        12

    accuracy                           0.47        17
   macro avg       0.31      0.33      0.32        17
weighted avg       0.43      0.47      0.45        17



### paretic RandomForest

In [25]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}


grid_search = GridSearchCV(RandomForestClassifier(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)

# 최적의 모델과 하이퍼파라미터 출력
best_rf_model = grid_search.best_estimator_
best_params = grid_search.best_params_
print("Best Random Forest Model:", best_rf_model)
print("Best Hyperparameters:", best_params)

# 최적의 모델로 테스트 데이터에 대한 예측 수행
y_pred_best_par_rf = best_rf_model.predict(X_test)

# 성능 메트릭 계산
conf_matrix_best = confusion_matrix(y_test, y_pred_best_par_rf)
print("Confusion Matrix:\n", conf_matrix_best)

accuracy_best_par_rf = accuracy_score(y_test, y_pred_best_par_rf)
print("Test Accuracy with Best Model:", accuracy_best_par_rf)

f1_best_par_rf = f1_score(y_test, y_pred_best_par_rf)
print("F1-score with Best Model:", f1_best_par_rf)

auc_best_par_rf = roc_auc_score(y_test, y_pred_best_par_rf)
print("AUC with Best Model:", auc_best_par_rf)

classification_rep_best_par_rf = classification_report(y_test, y_pred_best_par_rf)
print("Classification Report with Best Model:\n", classification_rep_best_par_rf)

Best Random Forest Model: RandomForestClassifier(min_samples_split=5, random_state=123)
Best Hyperparameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}
Confusion Matrix:
 [[ 0  5]
 [ 1 11]]
Test Accuracy with Best Model: 0.6470588235294118
F1-score with Best Model: 0.7857142857142857
AUC with Best Model: 0.4583333333333333
Classification Report with Best Model:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.69      0.92      0.79        12

    accuracy                           0.65        17
   macro avg       0.34      0.46      0.39        17
weighted avg       0.49      0.65      0.55        17



###SVM Linear


In [26]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear'],
    'degree':[2,3,4]
}


grid_search = GridSearchCV(SVC(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)


best_svm_model = grid_search.best_estimator_
best_params = grid_search.best_params_
print("Best SVM Model:", best_svm_model)
print("Best Hyperparameters:", best_params)


y_pred_best_par_svm_linear = best_svm_model.predict(X_test)

# 성능 메트릭 계산
conf_matrix_best = confusion_matrix(y_test, y_pred_best_par_svm_linear)
print("Confusion Matrix:\n", conf_matrix_best)

accuracy_best_par_svm_linear = accuracy_score(y_test, y_pred_best_par_svm_linear)
print("Test Accuracy with Best Model:", accuracy_best_par_svm_linear)

f1_best_par_svm_linear = f1_score(y_test, y_pred_best_par_svm_linear)
print("F1-score with Best Model:", f1_best_par_svm_linear)

y_scores_best_par_svm_linear = best_svm_model.decision_function(X_test)
auc_best_par_svm_linear = roc_auc_score(y_test, y_scores_best_par_svm_linear)
print("AUC with Best Model:", auc_best_par_svm_linear)

classification_rep_best_par_svm = classification_report(y_test, y_pred_best_par_svm_linear)
print("Classification Report with Best Model:\n", classification_rep_best_par_svm)

Best SVM Model: SVC(C=1, degree=2, kernel='linear', random_state=123)
Best Hyperparameters: {'C': 1, 'degree': 2, 'kernel': 'linear'}
Confusion Matrix:
 [[1 4]
 [5 7]]
Test Accuracy with Best Model: 0.47058823529411764
F1-score with Best Model: 0.6086956521739131
AUC with Best Model: 0.24999999999999997
Classification Report with Best Model:
               precision    recall  f1-score   support

           0       0.17      0.20      0.18         5
           1       0.64      0.58      0.61        12

    accuracy                           0.47        17
   macro avg       0.40      0.39      0.40        17
weighted avg       0.50      0.47      0.48        17



### SVM Poly

In [27]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['poly'],
    'degree': [2, 3, 4],
}


grid_search = GridSearchCV(SVC(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)


best_svm_model = grid_search.best_estimator_
best_params = grid_search.best_params_
print("Best SVM Model:", best_svm_model)
print("Best Hyperparameters:", best_params)


y_pred_best_par_svm_poly = best_svm_model.predict(X_test)

# 성능 메트릭 계산
conf_matrix_best = confusion_matrix(y_test, y_pred_best_par_svm_poly)
print("Confusion Matrix:\n", conf_matrix_best)

accuracy_best_par_svm_poly = accuracy_score(y_test, y_pred_best_par_svm_poly)
print("Test Accuracy with Best Model:", accuracy_best_par_svm_poly)

f1_best_par_svm_poly = f1_score(y_test, y_pred_best_par_svm_poly)
print("F1-score with Best Model:", f1_best_par_svm_poly)

y_scores_best_par_svm_poly = best_svm_model.decision_function(X_test)
auc_best_par_svm_poly = roc_auc_score(y_test, y_scores_best_par_svm_poly)
print("AUC with Best Model:", auc_best_par_svm_poly)

classification_rep_best_par_svm = classification_report(y_test, y_pred_best_par_svm_poly)
print("Classification Report with Best Model:\n", classification_rep_best_par_svm)

Best SVM Model: SVC(C=0.1, degree=2, kernel='poly', random_state=123)
Best Hyperparameters: {'C': 0.1, 'degree': 2, 'kernel': 'poly'}
Confusion Matrix:
 [[ 0  5]
 [ 0 12]]
Test Accuracy with Best Model: 0.7058823529411765
F1-score with Best Model: 0.8275862068965517
AUC with Best Model: 0.26666666666666666
Classification Report with Best Model:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.71      1.00      0.83        12

    accuracy                           0.71        17
   macro avg       0.35      0.50      0.41        17
weighted avg       0.50      0.71      0.58        17



### XGBoost

In [28]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb


param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2],
}


grid_search = GridSearchCV(xgb.XGBClassifier(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)


best_xgb_model = grid_search.best_estimator_
best_params = grid_search.best_params_
print("Best XGBoost Model:", best_xgb_model)
print("Best Hyperparameters:", best_params)


y_pred_best_par_xgb = best_xgb_model.predict(X_test)

conf_matrix_best = confusion_matrix(y_test, y_pred_best_par_xgb)
print("Confusion Matrix:\n", conf_matrix_best)

accuracy_best_par_xgb = accuracy_score(y_test, y_pred_best_par_xgb)
print("Test Accuracy with Best Model:", accuracy_best_par_xgb)

f1_best_par_xgb = f1_score(y_test, y_pred_best_par_xgb)
print("F1-score with Best Model:", f1_best_par_xgb)

auc_best_par_xgb = roc_auc_score(y_test, y_pred_best_par_xgb)
print("AUC with Best Model:", auc_best_par_xgb)

classification_rep_best_par_xgb = classification_report(y_test, y_pred_best_par_xgb)
print("Classification Report with Best Model:\n", classification_rep_best_par_xgb)

Best XGBoost Model: XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=123, ...)
Best Hyperparameters: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100}
Confusion Matrix:
 [[4 1]
 [3 9]]
Test Accuracy with Best Model: 0.7647058823529411
F1-score with Best Model: 0.8181818181818182
AUC with Best M